In [1]:
"""
Author: Zaw
Predicting the gain of smooth pursuit eye movement 
"""

import numpy as np
import pandas as pd
import math 
import time
import os
import sys
import warnings
warnings.simplefilter(action='ignore')
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from hyperopt import fmin,tpe,hp,STATUS_OK, Trials
import pickle
from hyperopt import hp
from sklearn.model_selection import GridSearchCV
#start = time.time()
import pickle

### Reading dataset

In [2]:
#data path
path = os.getcwd()
dirname = os.path.dirname(path)
data_file = 'data/data.csv'
data_path = os.path.join(dirname,data_file)

In [3]:
# load dataset
data = pd.read_csv(data_path)

In [4]:
data.head()

,Label,Patient,A1_1Deg,A1_1Deg_err,B1_1Deg,B1_1Deg_err,C1_1Deg,C1_1Deg_err,1_Deg_ResSoS_1,1Deg_AdjR2_1,...,8_Deg_ResSoS_1,8Deg_AdjR2_1,A2_8Deg,A2_8Deg_err,B2_8Deg,B2_8Deg_err,C2_8Deg,C2_8Deg_err,8_Deg_ResSoS_2,8Deg_AdjR2_2
0,Not_Parkinson's,PD001,10.388190,0.006132,0.049329,0.000006,0.011989,0.000337,8928.709194,0.990598,...,6698.838281,0.951413,10.273124,0.044206,0.386347,0.000384,0.037356,0.002439,6809.455800,0.939095
1,Not_Parkinson's,PD002,10.429055,0.006929,0.049478,0.000007,0.002837,0.000379,11388.604704,0.988092,...,4769.901822,0.967360,11.482500,0.049154,0.395412,0.000380,-0.002486,0.002419,8383.481840,0.939586
2,Not_Parkinson's,PD003,10.097088,0.005091,0.049368,0.000006,0.005707,0.000287,6140.292312,0.993141,...,3488.747176,0.968709,10.084338,0.019838,0.394595,0.000174,-0.035466,0.001106,1349.962240,0.987259
3,Not_Parkinson's,PD004,10.482881,0.022179,0.049585,0.000023,0.016056,0.001212,117748.979710,0.879372,...,10972.045844,0.920137,11.493414,0.048867,0.401386,0.000377,-0.042463,0.002395,8242.136178,0.940370
4,Not_Parkinson's,PD005,10.181273,0.008833,0.049368,0.000010,0.001569,0.000494,18447.340517,0.979692,...,18268.960666,0.809586,10.370348,0.036121,0.395392,0.000308,-0.030769,0.001960,4485.300264,0.960035


### Add calculated gain

In [5]:
OneDegGain = np.array([
                    0.912073221615219,
                    0.913615300167855,
                    0.946453363089275,
                    0.930324124079886,
                    1.02416020436668,
                    0.860239837548296,
                    0.964705222165739,
                    0.926124390992111,
                    0.968894500894799,
                    0.971483673217362,
                    0.944374366569671,
                    0.933778991410942,
                    0.890761283462127,

                    1.01702677636733,
                    1.01132254481336,
                    0.860332024178068,
                    0.898244125409108,
                    0.79226520992959,
                    0.86763213956782,

                    1.04294995675946,
                    0.954852019394754,
                    0.83139435252038,
                    0.969027023632338,
                    0.819025317130086,
                    1.59441787272771,
                    0.827558754578517,
                    0.882328237761475,
                    0.888806663106777  
                ])

TowDegGain  = np.array([
                    0.88795482315922,
                    0.85125583179904,
                    0.90557113971074,
                    0.839308531914961,
                    0.944624517823793,
                    0.936063494064383,
                    0.919761689613053,
                    0.857703024614198,
                    0.973218178159532,
                    0.936570806828469,
                    0.955653090017782,
                    0.943385867113339,
                    1.0186446733178,

                    0.957719524164782,
                    0.847863270306327,
                    0.762131387734656,
                    0.71762368625312,
                    0.699023420441996,
                    0.860669705999712,
                    
                    0.882514198813699,
                    0.900439093391636,
                    0.852845556682743,
                    0.662387043701962,
                    1.02691189892667,
                    0.929980500477329,
                    0.795054974778894,
                    0.915409616170048,
                    0.936562782215738
])

FourDegGain = np.array([
                    0.842657428787201,
                    0.927453824629543,
                    0.86099798858682,
                    0.826284460023473,
                    0.907866303422781,
                    0.886319852935139,
                    0.879363613650586,
                    0.945246226705543,
                    0.865730996104868,
                    0.949044695789533,
                    0.927054434038832,
                    0.907460103985577,
                    0.768451296309936,

                    0.670432552659765,
                    0.82906287014668,
                    0.792533491957787,
                    0.564497325152498,
                    0.566934462769341,
                    0.745104906289544,

                    0.863969643770036,
                    0.810695262798471,
                    0.665919934919724,
                    0.531701684005337,
                    0.804825842988611,
                    0.849537457619557,
                    0.843883703581647,
                    0.696788329556185,
                    0.878210689819177
])

SixDegGain = np.array([
                    0.795081992082097,
                    0.810824945533769,
                    0.776092810922058,
                    0.667632972614243,
                    0.875563679555973,
                    0.832291613200056,
                    0.896167465701247,
                    0.838934215329476,
                    0.91951999621913,
                    0.931875303042642,
                    0.881653294348028,
                    0.866711106710116,
                    0.950478365248271,

                    0.876172814217401,
                    0.793820437144166,
                    0.674619968989113,
                    0.508275303789709,
                    0.454613572040806,
                    0.761053355757645,

                    0.708138512300574,
                    0.875686044405767,
                    0.62197374329538,
                    0.682965128345167,
                    0.745383783235028,
                    0.904919560234923,
                    0.97349927736433,
                    0.790221320229067,
                    0.96896278814672
])

EightDegGain = np.array([
                    0.738482610210994,
                    0.986636423035039,
                    0.745501747649165,
                    0.757990700258398,
                    0.909797906001487,
                    0.781053010159343,
                    0.85865978762173,
                    0.859516710190982,
                    0.885411917285177,
                    0.828367438419201,
                    0.911980104842521,
                    0.863014297113752,
                    0.708399551278433,

                    0.673980441334396,
                    0.795542507563886,
                    0.795542507563886,
                    0.570216915379213,
                    0.470689666436208,
                    0.816745486005565,

                    0.730673411270229,
                    0.84023883503095,
                    0.634293750883143,
                    0.529398709071705,
                    0.750469831451345,
                    0.748589580457503,
                    0.881237281923916,
                    0.750773552799239,
                    0.972603229161408
])

In [6]:
# Array to dataframe
gain = pd.DataFrame({'OneDegGain':OneDegGain,'TowDegGain':TowDegGain,'FourDegGain':FourDegGain,
                    'SixDegGain':SixDegGain,'EightDegGain':EightDegGain})

In [7]:
# Concat with main dataframe
gain_df = pd.concat([data, gain], axis=1)

In [58]:
# Standardization
sc = StandardScaler()
data_input = pd.DataFrame(sc.fit_transform(data.iloc[:,2:]),index=data.iloc[:,2:].index, columns= data.iloc[:,2:].columns)

# Concat with gain
data_gain = pd.concat([data_input,gain],axis=1)

# # Features and labels
# x = data_input.iloc[:,2:]
# y = data.iloc[:,0]

# ## Convert the categorical variables to number
# LabelEncoder_gender = LabelEncoder()
# y = LabelEncoder_gender.fit_transform(y)

# # convert array to series 
# y = pd.Series(y,name='Label')
# df = pd.concat([x,y],axis=1)

In [59]:
data_gain

,A2_1Deg,B2_1Deg,C2_1Deg,1_Deg_ResSoS_2,1Deg_AdjR2_2,A1_2Deg,A1_2Deg_err,B1_2Deg,C1_2Deg,2Deg_AdjR2_1,...,A2_8Deg,B2_8Deg,C2_8Deg,8Deg_AdjR2_2,Label,OneDegGain,TowDegGain,FourDegGain,SixDegGain,EightDegGain
0,-0.093489,0.326950,-0.352989,-0.746190,0.797724,0.243010,-0.687114,-0.493465,0.442242,0.279247,...,0.080403,-1.933965,2.430309,0.335731,-1.035098,0.912073,0.887955,0.842657,0.795082,0.738483
1,-0.269396,0.069170,-0.150603,-0.406782,0.356029,0.262694,0.103664,0.376120,-0.034795,0.183586,...,2.038586,-0.663607,1.178616,0.340154,-1.035098,0.913615,0.851256,0.927454,0.810825,0.986636
2,-0.239333,-0.264076,-0.041293,-0.823263,0.887923,0.095490,-0.887982,-0.071083,0.018247,0.290520,...,-0.225273,-0.778159,0.142534,0.769440,-1.035098,0.946453,0.905571,0.860998,0.776093,0.745502
3,-0.107871,0.395914,1.761962,2.248299,-2.786026,0.867056,1.417576,2.265929,0.024783,-0.111163,...,2.056258,0.173552,-0.077294,0.347213,-1.035098,0.930324,0.839309,0.826284,0.667633,0.757991
4,-0.306015,0.125530,-0.707161,-0.621551,0.625289,0.091750,-0.113068,-0.439352,0.250819,0.215350,...,0.237825,-0.666471,0.290072,0.524289,-1.035098,1.024160,0.944625,0.907866,0.875564,0.909798
5,-0.105350,0.419347,0.392519,-0.221135,0.163252,0.209965,-0.840343,0.061339,-0.194388,0.288423,...,0.216055,1.420151,-0.399616,-0.380675,-1.035098,0.860240,0.936063,0.886320,0.832292,0.781053
6,-0.407542,-0.575147,-0.059279,-0.735383,0.766702,0.031561,-0.906895,-0.524409,0.024201,0.291263,...,-0.682914,-0.321981,-0.161340,0.705276,-1.035098,0.964705,0.919762,0.879364,0.896167,0.858660
7,-0.225167,0.596139,0.489860,0.109360,-0.275182,0.942725,0.972488,0.769743,-1.770690,0.022550,...,-2.642530,2.183482,-1.223308,-3.554912,-1.035098,0.926124,0.857703,0.945246,0.838934,0.859517
8,-0.367456,-1.198497,0.116359,-0.537493,0.506918,0.128696,-0.696935,-0.440762,-0.298726,0.279517,...,0.133028,-0.441392,-0.509905,0.738429,-1.035098,0.968895,0.973218,0.865731,0.919520,0.885412
9,0.051583,0.929254,0.028314,0.399384,-0.518398,0.371756,-0.201533,0.371028,-0.490793,0.231596,...,1.254033,0.948678,-0.100106,-0.897154,-1.035098,0.971484,0.936571,0.949045,0.931875,0.828367


In [60]:
#Check NaN
data_gain.isnull().values.any()

True

In [61]:
# Filling nan with mean values 
for col in data_gain:
    data_gain[col].fillna(data_gain[col].mean(),inplace=True)

In [64]:
#Check NaN again
data_gain.isnull().values.any()

False

In [63]:
data_gain.shape

(29, 47)

In [55]:
# Remove features with low variance (thrshold <= 0.05)
df_var = data_gain.var()
df_feat = list(df_var[df_var <= 0.05].index)
data_gain.drop(columns=df_feat, inplace=True)

In [56]:
data_gain.shape

(29, 42)

In [43]:
# Remove features with correlation(threshold > 0.95)
corr_matrix = data_gain.corr().abs()
mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
tri_df = corr_matrix.mask(mask)
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.95)]
data = data_gain.drop(to_drop,axis=1)

In [44]:
# Parameters
space = {'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
          'gamma': hp.uniform('gamma', 0, 0.2),
          'min_child_weight': hp.choice('min_child_weight', range(1, 6)),
          'subsample': hp.uniform('subsample', 0.7, 1.0),
          'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1.0),
          'max_depth': hp.choice('max_depth', range(3, 10)),
          'n_estimators': hp.choice('n_estimators', [50, 100, 200, 300, 400, 500, 1000])}
min_child_weight = range(1, 6)
max_depth = range(3, 10)
n_estimators= [100, 200, 300, 400, 500, 1000, 1500, 2000]

In [46]:
data_gain

,1_Deg_ResSoS_1,1Deg_AdjR2_1,A2_1Deg,B2_1Deg,C2_1Deg,1_Deg_ResSoS_2,1Deg_AdjR2_2,A1_2Deg,A1_2Deg_err,B1_2Deg,...,6Deg_AdjR2_2,A1_8Deg,B1_8Deg,C1_8Deg,8Deg_AdjR2_1,A2_8Deg,B2_8Deg,C2_8Deg,8Deg_AdjR2_2,Label
0,-0.496173,0.562528,-0.093489,0.326950,-0.352989,-0.746190,0.797724,0.243010,-0.687114,-0.493465,...,0.353269,1.692985,-0.676579,1.275180,0.388559,0.080403,-1.933965,2.430309,0.335731,-1.035098
1,-0.472873,0.529688,-0.269396,0.069170,-0.150603,-0.406782,0.356029,0.262694,0.103664,0.376120,...,0.419126,2.285951,-0.865028,1.074872,0.617965,2.038586,-0.663607,1.178616,0.340154,-1.035098
2,-0.522584,0.595850,-0.239333,-0.264076,-0.041293,-0.823263,0.887923,0.095490,-0.887982,-0.071083,...,0.449807,-0.196007,0.314260,-0.008476,0.637375,-0.225273,-0.778159,0.142534,0.769440,-1.035098
3,0.534541,-0.894932,-0.107871,0.395914,1.761962,2.248299,-2.786026,0.867056,1.417576,2.265929,...,0.195469,1.744291,1.142562,-1.015625,-0.061375,2.056258,0.173552,-0.077294,0.347213,-1.035098
4,-0.406015,0.419612,-0.306015,0.125530,-0.707161,-0.621551,0.625289,0.091750,-0.113068,-0.439352,...,0.185475,-1.456925,-0.628268,0.665786,-1.651711,0.237825,-0.666471,0.290072,0.524289,-1.035098
5,-0.334530,0.322090,-0.105350,0.419347,0.392519,-0.221135,0.163252,0.209965,-0.840343,0.061339,...,0.450146,0.194400,-0.374454,0.498465,0.202416,0.216055,1.420151,-0.399616,-0.380675,-1.035098
6,-0.443493,0.465576,-0.407542,-0.575147,-0.059279,-0.735383,0.766702,0.031561,-0.906895,-0.524409,...,0.461958,-0.800472,-0.916916,0.720835,0.884618,-0.682914,-0.321981,-0.161340,0.705276,-1.035098
7,1.086370,-1.960724,-0.225167,0.596139,0.489860,0.109360,-0.275182,0.942725,0.972488,0.769743,...,-0.499981,0.353103,-1.613532,2.185670,-0.775364,-2.642530,2.183482,-1.223308,-3.554912,-1.035098
8,-0.521509,0.595993,-0.367456,-1.198497,0.116359,-0.537493,0.506918,0.128696,-0.696935,-0.440762,...,0.393853,-0.291188,-0.925451,0.338709,0.976388,0.133028,-0.441392,-0.509905,0.738429,-1.035098
9,0.116433,-0.353601,0.051583,0.929254,0.028314,0.399384,-0.518398,0.371756,-0.201533,0.371028,...,0.447161,-0.132514,1.417857,-0.017192,-0.665145,1.254033,0.948678,-0.100106,-0.897154,-1.035098


In [45]:
# data set preparation
train , test = train_test_split(data_gain,train_size = 0.8,shuffle=True,random_state = 3)
#alidate , test = train_test_split(rest, train_size = 0.5, shuffle=True,random_state = 3)

# training set
data_tra_x = train.drop(['SixDegGain'],axis=1)
data_tra_y = train['SixDegGain']

# # validation set
# data_val_x = validate.drop(['OneDegGain'],axis=1)
# data_val_y = validate['OneDegGain']

# test set
data_tes_x = test.drop(['SixDegGain'],axis=1)
data_tes_y = test['SixDegGain']

KeyError: "['SixDegGain'] not found in axis"

In [78]:
# Define a objective function
def hyperparameter_tuning(params):
    model = XGBRegressor(**params,n_jobs=6,seed=1,random_state=1)
    model.fit(data_tra_x,data_tra_y)
    tra_preds = model.predict(data_tra_x)
    loss = mean_squared_error(data_tra_y,tra_preds,squared=False)
    return {'loss': loss,'status':STATUS_OK}
# Initialize trials object
trials = Trials()

# Fine tune the model
best_results = fmin(
    fn=hyperparameter_tuning,
    space = space,
    algo=tpe.suggest,
    max_evals= 50,
    trials=trials
)

100%|██████████| 50/50 [00:07<00:00,  7.11trial/s, best loss: 0.009600298997655726]


In [79]:
best_parameters = dict(best_results)

In [80]:
#save the parameters with lc
file_path = dirname+"/hyperparameters/xgb_reg_one.pkl"
pickle.dump(best_parameters, open(file_path,'wb'))

In [81]:
#load best parameters
file_path = dirname+"/hyperparameters/xgb_reg_one.pkl"
best_parameters_one= pickle.load(open(file_path,'rb'))

In [82]:
# best model single run
best_model = XGBRegressor(  n_estimators=n_estimators[best_parameters_one['n_estimators']], 
                            max_depth=max_depth[best_parameters_one['max_depth']],
                            min_child_weight=min_child_weight[best_parameters_one['min_child_weight']],
                            learning_rate=best_parameters_one['learning_rate'],
                            gamma=best_parameters_one['gamma'],
                            subsample=best_parameters_one['subsample'],
                            colsample_bytree=best_parameters_one['colsample_bytree'],
                            n_jobs=6, random_state=1, seed=1)

In [83]:
# Fit
best_model.fit(data_tra_x,data_tra_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9225060471177534, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=0.0005979664244952837, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.19997528546131876, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=400, n_jobs=6, num_parallel_tree=1, predictor='auto',
             random_state=1, reg_alpha=0, reg_lambda=1, ...)

In [84]:
  # training error
tra_pred = best_model.predict(data_tra_x)
tra_results = ['tra',mean_squared_error(data_tra_y,tra_pred),mean_squared_error(data_tra_y,tra_pred,squared=False),
            mean_absolute_error(data_tra_y,tra_pred),
            r2_score(data_tra_y,tra_pred)]

# # validation error
# val_pred = best_model.predict(data_val_x)
# val_results = ['val',mean_squared_error(data_val_y,val_pred),mean_squared_error(data_val_y,val_pred,squared=False),
#             mean_absolute_error(data_val_y,val_pred),
#             r2_score(data_val_y,val_pred)]
# testing error
tes_pred = best_model.predict(data_tes_x)
tes_results = ['tes',mean_squared_error(data_tes_y,tes_pred),mean_squared_error(data_tes_y,tes_pred,squared=False),
            mean_absolute_error(data_tes_y,tes_pred),
            r2_score(data_tes_y,tes_pred)]


all_set = tra_results,tes_results


all_set_df = pd.DataFrame(all_set,columns=['set','mse','rmse','mae','r2'])

In [85]:
all_set_df

,set,mse,rmse,mae,r2
0,tra,0.000093,0.009640,0.005875,0.999823
1,tes,2.222150,1.490688,1.337792,-0.279015


In [89]:
regressor = XGBRegressor(eval_metric = 'rmsle')
param_grid = {"max_depth":    [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

search = GridSearchCV(regressor, param_grid, cv=5).fit(data_tra_x, data_tra_y)

print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500}


In [90]:
regressor=XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                       n_estimators  = search.best_params_["n_estimators"],
                        max_depth     = search.best_params_["max_depth"],)

regressor.fit(data_tra_x,data_tra_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [91]:
predictions = regressor.predict(data_tes_x)

In [94]:
predictions

array([ 0.35391372, -1.116988  , -0.37252772, -0.24486302, -0.01570669,
       -0.18308266], dtype=float32)

In [98]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(data_tes_y, predictions) )
print("The score is %.5f" % RMSLE )

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [100]:
data

,B1_1Deg,C1_1Deg,1_Deg_ResSoS_1,1Deg_AdjR2_1,A2_1Deg,B2_1Deg,C2_1Deg,1_Deg_ResSoS_2,1Deg_AdjR2_2,A1_2Deg,...,A2_8Deg,B2_8Deg,C2_8Deg,8Deg_AdjR2_2,OneDegGain,TowDegGain,FourDegGain,SixDegGain,EightDegGain,Label
0,-0.076286,0.187603,-0.496173,0.562528,-0.093489,0.326950,-0.352989,-0.746190,0.797724,0.243010,...,0.080403,-1.933965,2.430309,0.335731,-2.307471e-01,5.905165e-02,3.070946e-01,-3.363555e-02,-3.295803e-01,0
1,0.451166,-0.394886,-0.472873,0.529688,-0.269396,0.069170,-0.150603,-0.406782,0.356029,0.262694,...,2.038586,-0.663607,1.178616,0.340154,-2.197516e-01,-3.568976e-01,1.046298e+00,8.906335e-02,1.718203e+00,0
2,0.063202,-0.212194,-0.522584,0.595850,-0.239333,-0.264076,-0.041293,-0.823263,0.887923,0.095490,...,-0.225273,-0.778159,0.142534,0.769440,1.439370e-02,2.587164e-01,4.669764e-01,-1.816352e-01,-2.716579e-01,0
3,0.827975,0.446425,0.534541,-0.894932,-0.107871,0.395914,1.761962,2.248299,-2.786026,0.867056,...,2.056258,0.173552,-0.077294,0.347213,-1.006126e-01,-4.923092e-01,1.643650e-01,-1.026960e+00,-1.685982e-01,0
4,0.060480,-0.475598,-0.406015,0.419612,-0.306015,0.125530,-0.707161,-0.621551,0.625289,0.091750,...,0.237825,-0.666471,0.290072,0.524289,5.684670e-01,7.013504e-01,8.755459e-01,5.936301e-01,1.084126e+00,0
5,-0.237615,0.680540,-0.334530,0.322090,-0.105350,0.419347,0.392519,-0.221135,0.163252,0.209965,...,0.216055,1.420151,-0.399616,-0.380675,-6.003348e-01,6.043191e-01,6.877171e-01,2.563723e-01,2.171367e-02,0
6,-0.502797,-0.344210,-0.443493,0.465576,-0.407542,-0.575147,-0.059279,-0.735383,0.766702,0.031561,...,-0.682914,-0.321981,-0.161340,0.705276,1.445350e-01,4.195532e-01,6.270768e-01,7.542138e-01,6.621303e-01,0
7,-0.461836,2.725447,1.086370,-1.960724,-0.225167,0.596139,0.489860,0.109360,-0.275182,0.942725,...,-2.642530,2.183482,-1.223308,-3.554912,-1.305580e-01,-2.838246e-01,1.201401e+00,3.081440e-01,6.692017e-01,0
8,-0.378231,-0.430705,-0.521509,0.595993,-0.367456,-1.198497,0.116359,-0.537493,0.506918,0.128696,...,0.133028,-0.441392,-0.509905,0.738429,1.744058e-01,1.025433e+00,5.082359e-01,9.362210e-01,8.828908e-01,0
9,2.311421,-1.533515,0.116433,-0.353601,0.051583,0.929254,0.028314,0.399384,-0.518398,0.371756,...,1.254033,0.948678,-0.100106,-0.897154,1.928674e-01,6.100690e-01,1.234514e+00,1.032517e+00,4.121557e-01,0
